<a href="https://colab.research.google.com/github/TonyIssacJames/eva04_jan_2020_public/blob/master/S04/EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 11):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 1.8770, Accuracy: 2875/10000 (29%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 1.8694, Accuracy: 2976/10000 (30%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.7294, Accuracy: 7905/10000 (79%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.7271, Accuracy: 7916/10000 (79%)



loss=0.6775316596031189 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 36.59it/s]



Test set: Average loss: 0.7223, Accuracy: 7929/10000 (79%)



## Trying our network now

In [7]:
class Net_Mine(nn.Module):
    def __init__(self):
        super(Net_Mine, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #input -? OUtput? RF
        self.bn1   = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 28, 3, padding=1)
        self.bn2   = nn.BatchNorm2d(28)
        self.pool1 = nn.MaxPool2d(2, 2)#14x14
        self.conv3 = nn.Conv2d(28, 20, 3, padding=1)
        self.bn3   = nn.BatchNorm2d(20)
        self.conv4 = nn.Conv2d(20, 27, 3, padding=1)
        self.bn4   = nn.BatchNorm2d(27)
        self.pool2 = nn.MaxPool2d(2, 2) #7x7
        self.conv5 = nn.Conv2d(27, 16, 3)
        self.bn5   = nn.BatchNorm2d(16)
        self.conv6 = nn.Conv2d(16, 10, 3)
        self.bn6   = nn.BatchNorm2d(10)
        #self.conv7 = nn.Conv2d(1024, 10, 3)
        self.avg_pool = nn.AvgPool2d(3)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn2(self.conv2(F.relu(self.bn1(self.conv1(x)))))))
        x = self.pool2(F.relu(self.bn4(self.conv4(F.relu(self.bn3(self.conv3(x)))))))
        x = F.relu(self.bn6(self.conv6(F.relu(self.bn5(self.conv5(x))))))
        x = F.relu(self.avg_pool(x))
        #x = self.avg_pool(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

model_mine = Net_Mine().to(device)
summary(model_mine, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
            Conv2d-3           [-1, 28, 28, 28]           4,060
       BatchNorm2d-4           [-1, 28, 28, 28]              56
         MaxPool2d-5           [-1, 28, 14, 14]               0
            Conv2d-6           [-1, 20, 14, 14]           5,060
       BatchNorm2d-7           [-1, 20, 14, 14]              40
            Conv2d-8           [-1, 27, 14, 14]           4,887
       BatchNorm2d-9           [-1, 27, 14, 14]              54
        MaxPool2d-10             [-1, 27, 7, 7]               0
           Conv2d-11             [-1, 16, 5, 5]           3,904
      BatchNorm2d-12             [-1, 16, 5, 5]              32
           Conv2d-13             [-1, 10, 3, 3]           1,450
      BatchNorm2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [8]:
model_mine = Net_Mine().to(device)
optimizer  = optim.SGD(model_mine.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model_mine, device, train_loader, optimizer, epoch)
    test(model_mine, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0818, Accuracy: 9874/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0636, Accuracy: 9870/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0248, Accuracy: 9920/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0260, Accuracy: 9917/10000 (99%)



loss=0.00788109004497528 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 41.55it/s]



Test set: Average loss: 0.0263, Accuracy: 9921/10000 (99%)

